# Using Xarray-Spatial and Datashader to Quantify Pharmacy Deserts

In [18]:
import datashader as ds
import numpy as np
import pandas as pd
import geopandas as gpd

from datashader.transfer_functions import stack
from datashader.transfer_functions import shade
from datashader.transfer_functions import set_background
from datashader.colors import inferno

from xrspatial.classify import natural_breaks
from xrspatial.classify import binary
from xrspatial import proximity

from spatialpandas import GeoDataFrame

## Load data

In [19]:
# Load Pharmacies and add out x, y fields based on CalcLocation
pharmacy_df = pd.read_csv(r"C:\Users\sapir\Downloads\facility.csv")
coords = pharmacy_df['CalcLocation'].str.split(',', expand=True)
pharmacy_df['y'] = np.array(coords[0], dtype='float64')
pharmacy_df['x'] = np.array(coords[1], dtype='float64')

In [20]:
# Load Census Block Groups and Calculate Percent over 65 years-old
blockgroup_df = GeoDataFrame(gpd.read_file(r"C:\Users\sapir\Downloads\USA_Block_Groups-shp\USA_Block_Groups.shp"))
blockgroup_df['ABOVE_65'] = blockgroup_df[['AGE_65_74', 'AGE_75_84', 'AGE_85_UP']].sum(axis=1) 
blockgroup_df['PCT_ABOVE_65'] = blockgroup_df['ABOVE_65'] / blockgroup_df['POP2010']

In [21]:
#del blockgroup_df['OBJECT_1']
blockgroup_df

,OBJECTID_1,ObjectID,STATE_FIPS,CNTY_FIPS,STCOFIPS,TRACT,BLKGRP,FIPS,POP2010,POP10_SQMI,...,FAMILIES,AVE_FAM_SZ,HSE_UNITS,VACANT,OWNER_OCC,RENTER_OCC,SQMI,geometry,ABOVE_65,PCT_ABOVE_65
0,1,0,48,167,48167,724101,1,481677241011,477,132.5,...,97,2.57,799,550,78,171,3.60,"MultiPolygon([[[-94.7372579289103, 29.32428411...",140,0.293501
1,2,1,48,245,48245,000600,6,482450006006,1260,6000.0,...,317,3.46,472,45,262,165,0.21,"MultiPolygon([[[-94.1277140349747, 30.12067592...",76,0.060317
2,3,2,48,167,48167,724300,5,481677243005,604,7550.0,...,123,3.07,424,139,87,198,0.08,"MultiPolygon([[[-94.7732350204798, 29.30697297...",60,0.099338
3,4,3,48,071,48071,710200,3,480717102003,5276,150.0,...,1455,3.41,1709,76,1548,85,35.18,"MultiPolygon([[[-94.7161589478771, 29.72786202...",218,0.041319
4,5,4,48,167,48167,724200,2,481677242002,906,11325.0,...,186,2.81,604,120,74,410,0.08,"MultiPolygon([[[-94.7696420580345, 29.31721401...",70,0.077263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219824,219827,219826,26,115,26115,990000,0,261159900000,0,0.0,...,0,0.00,0,0,0,0,0.19,"MultiPolygon([[[-83.3855070111396, 41.87580502...",0,NaN
219825,219828,219827,72,033,72033,990201,0,720339902010,-99,-99.0,...,-99,-99.00,-99,-99,-99,-99,0.00,"MultiPolygon([[[-66.1310319480659, 18.44882092...",-297,3.000000
219826,219829,219828,12,057,12057,990000,0,120579900000,0,0.0,...,0,0.00,0,0,0,0,1.79,"MultiPolygon([[[-82.4328329499806, 27.82253797...",0,NaN
219827,219830,219829,37,019,37019,990100,0,370199901000,0,0.0,...,0,0.00,0,0,0,0,0.56,"MultiPolygon([[[-78.4868539895199, 33.87145592...",0,NaN


In [22]:
# Load Census County and Calculate Percent over 65 years-old
county_df = GeoDataFrame(gpd.read_file(r"C:\Users\sapir\Downloads\USA_Counties-shp\USA_Counties.shp"))
county_df['ZONE_ID'] = county_df['OBJECTID'].astype(np.int16)
county_df

C:\programs\Anaconda3\envs\pharmacy-deserts\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'coun' is not defined

## Define Study Area

In [ ]:
x_range = (-124.848974, -66.885444)
y_range = (24.396308, 49.384358)

W = 1600
H = 800

cvs = ds.Canvas(plot_width=W, plot_height=H,
                x_range=x_range, y_range=y_range)

county_mask = cvs.polygons(county_df, geometry='geometry')
set_background(shade(county_mask, cmap='#333333', alpha=255), 'black')

### Create a "Distance to Nearest Pharmacy" Layer & Classify into 5 Groups

In [ ]:
pharmacy_raster = cvs.points(pharmacy_df, 'x', 'y')
proximity_raster = proximity(pharmacy_raster, distance_metric='GREAT_CIRCLE').where(county_mask)
proximity_raster.data[~np.isfinite(proximity_raster.data)] = 0.0

proximity_classifed = natural_breaks(proximity_raster, k=5).where(county_mask)

image_pharmacy = shade(proximity_classifed, cmap=inferno, alpha=255)
image_pharmacy = set_background(image_pharmacy, 'black')
#image_pharmacy

### Create an Age Layer  & Classify into 5 Groups

In [ ]:
age_raster = cvs.polygons(blockgroup_df, geometry='geometry', agg=ds.mean('PCT_ABOVE_65'))
age_raster.data[~np.isfinite(age_raster.data)] = 0.0
age_classifed = natural_breaks(age_raster, k=5).where(county_mask)

age_image = shade(age_classifed, cmap=inferno, alpha=255)
age_image = set_background(age_image, 'black')
age_image

### Combine layers to highlight seniors at risk from pharmacy deserts

In [ ]:
pharmacy_deserts = binary(proximity_classifed, [4.0])
older_regions = binary(age_classifed, [4.0])
target_deserts = (pharmacy_deserts * older_regions).where(county_mask)
target_deserts_img = shade(target_deserts, cmap=['#333333', 'fuchsia'], alpha=255, how='linear')
set_background(target_deserts_img, 'black')

###  Summarize seniors at risk by county

In [ ]:
from datashader.colors import Set1

counties_raster = cvs.polygons(county_df, geometry='geometry', agg=ds.max('OBJECTID'))
counties_image = shade(counties_raster, cmap=Set1, alpha=225, how='linear')
set_background(counties_image, 'black')

## Zonal Statistics

Zonal statistics allows for calculating summary statistics for specific areas or zones within a datashader aggregate. Zones are defined by creating an integer aggregate where the cell values are zone_ids. The output of zonal statistics is a Pandas dataframe containing summary statistics for each zone based on a value raster.


In [ ]:
from xrspatial import zonal_stats

# summary functions
zonal_funcs = dict(pharmacy_desert_mean=lambda z: z.mean())

# zones
counties_raster.data = counties_raster.data.astype(np.int64)

# values to summarize
target_deserts.data = target_deserts.data.astype(np.int8)

# execute summary functions on each zone and take top 10
results = zonal_stats(counties_raster, target_deserts, zonal_funcs)

### Join result back to counties layer

In [ ]:

cols = ['pharmacy_desert_mean', 'NAME', 'STATEFP', 'geometry']
final_df = pd.merge(county_df, results, left_on='ZONE_ID', right_index=True)[cols]
final_df.nlargest(10, 'pharmacy_desert_mean')

In [ ]:
from xrspatial import hillshade

counties_raster = cvs.polygons(county_df, geometry='geometry', agg=ds.max('OBJECTID'))

desert_raster = cvs.polygons(final_df.nlargest(10, 'pharmacy_desert_mean'),
                             geometry='geometry',
                             agg=ds.mean('pharmacy_desert_mean'))

county_mask = cvs.polygons(county_df, geometry='geometry')

In [ ]:
img = stack(
    shade(county_mask, cmap=['#333333'], alpha=255),
    shade(counties_raster, cmap=['#333333', '#ffffff'], alpha=25),
    shade(desert_raster, cmap=['#333333','#ff0000'], alpha=200),
    shade(hillshade(desert_raster), cmap=['#333333', '#ff0000'], alpha=100),
)
set_background(img, 'black')